In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
from data.pointCld_dataset import PointCld_dataset
from training import *
from PolyhedronGenerator.polyhedronGenerator import Point


In [ ]:
'Set an experiment name. Uncomment delete_experiment, if path exists and shall be overwritten.'
path = 'Clean_Training'
delete_experiment(path)
params = create_experiment(path)
'Uncomment, if existing experiment shall be continued. Give path of existing experiment as input.'
#params = create_retrain_experiment('data/network_data/' + path[:-len('_ContAfter7')] + '/epoch_7_trained_model.pth')
cfg_path=params["cfg_path"]

In [ ]:
'Generate new datasets if dataset_name does not exist. Otherwise, use existing one.'
train_dataset = PointCld_dataset(dataset_name='clean_polyhedra_train_500',
                                    size=500, mode=Mode.TRAIN, cfg_path=cfg_path, augmentation=None, batch_size=1)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                               batch_size=1,
                                               shuffle=True, num_workers=4)

validate_dataset = PointCld_dataset(dataset_name='clean_polyhedra_validate_50', size=50, cfg_path=cfg_path, mode=Mode.TRAIN, seed=7, batch_size=1)
validate_loader = torch.utils.data.DataLoader(dataset=validate_dataset,
                                              batch_size=1,
                                              shuffle=False, num_workers=4)

test_dataset = PointCld_dataset(dataset_name='clean_polyhedra_test_50', size=50, cfg_path=cfg_path, mode=Mode.TEST, seed=105, batch_size=1)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                              batch_size=1,
                                              shuffle=False, num_workers=4)

trainer = Training(cfg_path, cuda_device_id=0)

In [ ]:
'Execute training.'
optimiser_params = {'lr': 1e-4}
nr_voxels_per_dim = 21
wr = 8

trainer.setup_model(model=Network, convModel = 'resnet', weight_ratio=wr,
                    optimiser=torch.optim.Adam, optimiser_params=optimiser_params,
                    loss_function=nn.CrossEntropyLoss, nr_voxels_per_dim=nr_voxels_per_dim, batch_size=500)

trainer.execute_training(train_loader, validate_loader, num_epochs=13, do_preprocessing=True, validate_frequency=1)